In [2]:
import pandas as pd

team_recs = pd.read_csv('Team_Records.csv')
print(team_recs)

       Season   Lg                 Team   W   L   W/L%  Finish    SRS  Pace  \
0     2017-18  NBA       Boston Celtics  29  10  0.744       1   4.38  95.4   
1     2016-17  NBA      Boston Celtics*  53  29  0.646       1   2.25  96.8   
2     2015-16  NBA      Boston Celtics*  48  34  0.585       2   2.84  98.5   
3     2014-15  NBA      Boston Celtics*  40  42  0.488       2  -0.40  95.8   
4     2013-14  NBA       Boston Celtics  25  57  0.305       4  -4.97  93.3   
...       ...  ...                  ...  ..  ..    ...     ...    ...   ...   
1478  1999-00  NBA  Vancouver Grizzlies  22  60  0.268       7  -5.10  91.0   
1479  1998-99  NBA  Vancouver Grizzlies   8  42  0.160       7  -8.94  89.0   
1480  1997-98  NBA  Vancouver Grizzlies  19  63  0.232       6  -7.47  92.1   
1481  1996-97  NBA  Vancouver Grizzlies  14  68  0.171       7 -10.17  88.4   
1482  1995-96  NBA  Vancouver Grizzlies  15  67  0.183       7  -9.55  91.0   

      Rel_Pace   ORtg  Rel_ORtg   DRtg  Rel_DRtg   